Importing Libraries

In [1]:
import pandas as pd
import numpy as np

Importing the Data

In [2]:
df = pd.read_csv('car_data_bulk.csv', low_memory=False)
df.set_index('bulk_id')

,car_reg_no,vehicle_speed,heading,distance,altitude,o_s1_b1_voltage,o_s2_b2_voltage,o_s1_current,calculated_engine_load,engine_rpm,...,egr_error,absolute_barometric_pressure,engine_coolant_temperature,intake_air_temperature,catalyst_temperature_b1_s1,catalyst_temperature_b1_s2,ambient_air_temperature,time_stamp,latitude,longitude
bulk_id,,,,,,,,,,,,,,,,,,,,,
1,45.0,80.0,0.0,0.0,-40.0,0,0,0,0,0.0,...,0.0000,0,0.0,0.0,NaN,0.0,0.0,0,2020-08-15,24.8828713
2,45.0,80.0,0.0,0.0,-40.0,0,0,0,0,0.0,...,0.0000,0,0.0,0.0,NaN,0.0,0.0,0,2020-08-15,24.8828713
3,45.0,80.0,0.0,0.0,-40.0,0,0,0,0,0.0,...,0.0000,0,0.0,0.0,NaN,0.0,0.0,0,2020-08-15,24.8828713
4,45.0,80.0,0.0,0.0,-40.0,0,0,0,0,0.0,...,0.0000,0,0.0,0.0,NaN,0.0,0.0,0,2020-08-15,24.8828713
5,45.0,80.0,0.0,0.0,-40.0,0,0,0,0,0.0,...,0.0000,0,0.0,0.0,NaN,0.0,0.0,0,2020-08-15,24.8828713
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4040506,176.0,86.0,0.0,NaN,0.0,NaN,0.6700,0.0000,100.0000,2252.0,...,50.1961,0.0000,84.0,34.0,NaN,NaN,NaN,2021-12-06 20:25:50,0.000000,0.000000
4040507,176.0,86.0,0.0,NaN,0.0,NaN,0.6700,0.0000,100.0000,2252.0,...,50.1961,0.0000,84.0,34.0,NaN,NaN,NaN,2021-12-06 20:25:52,0.000000,0.000000
4040508,176.0,86.0,0.0,NaN,0.0,NaN,0.6700,0.0000,100.0000,2252.0,...,50.1961,0.0000,84.0,34.0,NaN,NaN,NaN,2021-12-06 20:25:54,0.000000,0.000000


Filtering the car reg no. 113 data

In [3]:
df = df.loc[df['car_reg_no'] == 113]

Dropping the null columns

In [4]:
df = df.dropna(axis=1, how='all') #dropping the columns which contains all null values

# #repeated columns which one to keep?
print(df['o_s1_b1_fuel_air_equivalence_ratio'].describe())
print(df['o_s1_b1_fuelair_equivalence_ratio'].describe()) #contains all zeros so removing it

count    865803.000000
mean         14.244530
std           0.858923
min           6.725500
25%          13.848100
50%          14.700000
75%          14.700000
max          15.234700
Name: o_s1_b1_fuel_air_equivalence_ratio, dtype: float64
count    865429.0
mean          0.0
std           0.0
min           0.0
25%           0.0
50%           0.0
75%           0.0
max           0.0
Name: o_s1_b1_fuelair_equivalence_ratio, dtype: float64


Dropping repeated column

In [5]:
df = df.drop('o_s1_b1_fuelair_equivalence_ratio',axis=1)

Removing null values

In [6]:
print(df.isna().sum())

#finding the pattern in null values of time_stamp
print(df[df['time_stamp'].isna()])

cond = df['time_stamp'].isna()

df.loc[cond ,'time_stamp'] = df.loc[cond , 'longitude'] #observed that when time_stamp was null, time_stamp was actually present in the longitude column so replacing it

df.loc[cond, 'longitude'] = np.nan

df.dropna(subset=['time_stamp'])

df['engine_rpm'] = df['engine_rpm'].fillna(0) #filling the null values with zero as we observed that when engine_rpm was null vehicle_speed was 0/null and other values were also null, so the engine would be turned off

df[df['engine_rpm']==0] = df[df['engine_rpm']==0].fillna(0) #when engine is turned off all other components are also turned off

print('Data after filling the engine off readings')
print(df.isna().sum())

bulk_id                                    0
car_reg_no                                 0
vehicle_speed                              1
heading                                    1
altitude                                   1
o_s2_b2_voltage                       105675
o_s1_current                           79110
calculated_engine_load                 82489
engine_rpm                             82860
spark_advance                          83675
absolute_load_value                    84298
throttle_position                      84409
relative_throttle_position             84544
absolute_throttle_position             84729
ap_pos_d                               84962
ap_pos_e                               85179
commanded_evaporative_purge            85333
o_s1_b1_fuel_air_equivalence_ratio     85659
short_term_fuel_trim_b1                32486
long_term_fuel_trim_b1                 32486
intake_manifold_absolute_pressure     951452
mass_air_flow_rate                    111094
absolute_b

Standardizing Time Stamp Formats

In [7]:
def generate_interval(data):
    time = data['time_stamp']
    n = len(time)
    base_time = pd.to_datetime(time.iloc[0], format='%m/%d/%Y %H:%M') + pd.Timedelta(seconds=1)
    intervals = [base_time + pd.Timedelta(seconds=i * 2) for i in range(n)]

    data['time_stamp'] = intervals
    return data

df_seg1 = df.iloc[0:707198].copy()
df_seg2 = df.iloc[707198:950133].copy()
df_seg3 = df.iloc[950133:].copy()


print('Segment 1: ',df_seg1['time_stamp'].unique())
print('Segment 2: ',df_seg2['time_stamp'].unique())
print('Segment 3: ',df_seg3['time_stamp'].unique())

df_seg1['time_stamp'] = pd.to_datetime(df_seg1['time_stamp'])
df_seg3['time_stamp'] = pd.to_datetime(df_seg3['time_stamp'])

df_seg2 = df_seg2.groupby('time_stamp').apply(generate_interval)
df_seg2 = df_seg2.reset_index(level = 'time_stamp', drop=True)
print(df_seg2.head(10))

df = pd.concat([df_seg1,df_seg2,df_seg3])


Segment 1:  [0 '2021-02-09T08:00:16' '2021-02-09T08:00:18' ... '2022-01-21 13:30:55'
 '2022-01-21 13:30:57' '2022-01-21 13:30:59']
Segment 2:  ['10/10/2021 6:25' '10/10/2021 6:26' '10/10/2021 6:27' '10/10/2021 6:28'
 '10/10/2021 6:29' '10/10/2021 6:30' '10/10/2021 6:31' '10/10/2021 6:32'
 '10/10/2021 6:33' '10/10/2021 6:34' '10/10/2021 6:35' '10/10/2021 6:36'
 '10/10/2021 6:37' '10/10/2021 6:38' '10/10/2021 6:39' '10/10/2021 6:40'
 '10/10/2021 6:41' '10/10/2021 6:42' '10/10/2021 6:43' '10/10/2021 6:44'
 '10/10/2021 6:45' '10/10/2021 6:46' '10/10/2021 6:47' '10/10/2021 6:48'
 '10/10/2021 6:49' '10/10/2021 6:50' '10/10/2021 6:51' '10/10/2021 6:52'
 '10/10/2021 6:53' '10/10/2021 6:54' '10/10/2021 6:55' '10/10/2021 6:56'
 '10/10/2021 6:57' '10/10/2021 6:58' '10/10/2021 6:59' '10/10/2021 7:00'
 '10/10/2021 7:01' '10/10/2021 7:02' '10/10/2021 7:03' '10/10/2021 7:04'
 '10/10/2021 7:05' '10/10/2021 7:06' '10/10/2021 7:07' '10/10/2021 7:08'
 '10/10/2021 7:09' '10/10/2021 7:10' '10/10/2021 7:11'

C:\Users\Moiz\AppData\Local\Temp\ipykernel_7680\154980012.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_seg2 = df_seg2.groupby('time_stamp').apply(generate_interval)


         bulk_id  car_reg_no  vehicle_speed  heading  altitude  \
2462639  2462640       113.0            0.0      0.0       0.0   
2462640  2462641       113.0           14.0      0.0       0.0   
2462641  2462642       113.0            7.0      0.0       0.0   
2462642  2462643       113.0           11.0      0.0       0.0   
2462643  2462644       113.0           11.0      0.0       0.0   
2462644  2462645       113.0           16.0      0.0       0.0   
2462645  2462646       113.0           18.0      0.0       0.0   
2462646  2462647       113.0           23.0      0.0       0.0   
2462647  2462648       113.0           29.0      0.0       0.0   
2462648  2462649       113.0           35.0      0.0       0.0   

        o_s2_b2_voltage o_s1_current calculated_engine_load  engine_rpm  \
2462639               0            0                26.6667      1332.0   
2462640               0            0                26.6667      1332.0   
2462641           0.035            0            

Removing null value other than engine off values

In [8]:
df['latitude'] = df['latitude'].fillna(0).astype('float')
df['longitude'] = df['longitude'].fillna(0).astype('float')

Correcting column data types

In [9]:
df['o_s2_b2_voltage'] = df['o_s2_b2_voltage'].astype('float')
df['o_s1_current'] = df['o_s1_current'].astype('float')
df['calculated_engine_load'] = df['calculated_engine_load'].astype('float')
df['absolute_load_value'] = df['absolute_load_value'].astype('float')
df['throttle_position'] = df['throttle_position'].astype('float')
df['relative_throttle_position'] = df['relative_throttle_position'].astype('float')
df['ap_pos_d'] = df['ap_pos_d'].astype('float')
df['ap_pos_e'] = df['ap_pos_e'].astype('float')
df['commanded_evaporative_purge'] = df['commanded_evaporative_purge'].astype('float')
df['short_term_fuel_trim_b1'] = df['short_term_fuel_trim_b1'].astype('float')
df['absolute_barometric_pressure'] = df['absolute_barometric_pressure'].astype('float')

Removing remaining null values

In [10]:
df.fillna(df[df!=0].mean(),inplace=True) #replacing null values with mean value

print(df['o_s1_current'].unique()) #all zeros
print(df['catalyst_temperature_b1_s1'].unique()) #all zeros
print(df['catalyst_temperature_b1_s2'].unique()) #all zeros

# #dropping the columns
df = df.drop(['o_s1_current','catalyst_temperature_b1_s1','catalyst_temperature_b1_s2'], axis=1)

print('\nCleaned Data:')
print(df.isna().sum())
print('No. of rows:',len(df))

[ 0. nan]
[ 0. nan]
[ 0. nan]

Cleaned Data:
bulk_id                               0
car_reg_no                            0
vehicle_speed                         0
heading                               0
altitude                              0
o_s2_b2_voltage                       0
calculated_engine_load                0
engine_rpm                            0
spark_advance                         0
absolute_load_value                   0
throttle_position                     0
relative_throttle_position            0
absolute_throttle_position            0
ap_pos_d                              0
ap_pos_e                              0
commanded_evaporative_purge           0
o_s1_b1_fuel_air_equivalence_ratio    0
short_term_fuel_trim_b1               0
long_term_fuel_trim_b1                0
intake_manifold_absolute_pressure     0
mass_air_flow_rate                    0
absolute_barometric_pressure          0
engine_coolant_temperature            0
intake_air_temperature             

Filtering for 2021 data only

In [11]:
df = df.sort_values(by='time_stamp')
start = pd.to_datetime('2021-01-01 00:00:00')
end = pd.to_datetime('2021-12-31 23:59:59')

df = df[(df['time_stamp'] >= start) & (df['time_stamp']<=end)]

print(df['time_stamp'].iloc[0])
print(df['time_stamp'].iloc[-1])
print('No. of rows:',len(df))

2021-02-05 11:05:07
2021-12-31 20:55:16
No. of rows: 906013


Checking for duplicate values

In [12]:
duplicate = {}
for column in df.columns:
    duplicate[column] = df[column].duplicated()

duplicate = pd.DataFrame(duplicate)
print('No. of duplicate values in each column')
print(duplicate.sum())

No. of duplicate values in each column
bulk_id                                    0
car_reg_no                            906012
vehicle_speed                         905878
heading                               894697
altitude                              895312
o_s2_b2_voltage                       905949
calculated_engine_load                905802
engine_rpm                            903393
spark_advance                         905862
absolute_load_value                   905852
throttle_position                     905857
relative_throttle_position            905875
absolute_throttle_position            905856
ap_pos_d                              905907
ap_pos_e                              905897
commanded_evaporative_purge           905766
o_s1_b1_fuel_air_equivalence_ratio    905438
short_term_fuel_trim_b1               905953
long_term_fuel_trim_b1                905982
intake_manifold_absolute_pressure     906008
mass_air_flow_rate                    903646
absolute_baromet

Analyzing duplicated time stamp data

In [13]:
df[df.duplicated(subset=['time_stamp'], keep=False)].iloc[3000:3010] #checking for random bunch of values
#the duplicated entries are duplicate for all columns except bulk_id we can ignore bulk_id as it is just used for indexing


,bulk_id,car_reg_no,vehicle_speed,heading,altitude,o_s2_b2_voltage,calculated_engine_load,engine_rpm,spark_advance,absolute_load_value,...,long_term_fuel_trim_b1,intake_manifold_absolute_pressure,mass_air_flow_rate,absolute_barometric_pressure,engine_coolant_temperature,intake_air_temperature,ambient_air_temperature,time_stamp,latitude,longitude
574270,574271,113.0,3.0,240.514435,-22.4856,0.135,57.2549,832.0,52.9412,33.0,...,1.5625,32.0,3.51,100.0,91.0,50.0,35.0,2021-04-06 14:10:39,24.909687,67.079369
586219,586220,113.0,3.0,240.514435,-22.4856,0.135,57.2549,832.0,52.9412,33.0,...,1.5625,32.0,3.51,100.0,91.0,50.0,35.0,2021-04-06 14:10:39,24.909687,67.079369
574272,574273,113.0,5.0,240.514435,-22.4856,0.135,60.0000,1017.0,60.7843,38.0,...,1.5625,32.0,3.64,100.0,91.0,50.0,35.0,2021-04-06 14:10:41,24.909687,67.079369
586223,586224,113.0,5.0,240.514435,-22.4856,0.135,60.0000,1017.0,60.7843,38.0,...,1.5625,32.0,3.64,100.0,91.0,50.0,35.0,2021-04-06 14:10:41,24.909687,67.079369
586227,586228,113.0,6.0,240.514435,-22.4856,0.135,54.9020,1139.0,53.3333,22.0,...,1.5625,32.0,3.59,100.0,91.0,50.0,35.0,2021-04-06 14:10:43,24.909687,67.079369
574274,574275,113.0,6.0,240.514435,-22.4856,0.135,54.9020,1139.0,53.3333,22.0,...,1.5625,32.0,3.59,100.0,91.0,50.0,35.0,2021-04-06 14:10:43,24.909687,67.079369
586231,586232,113.0,8.0,240.514435,-22.4856,0.250,61.1765,1345.0,69.4118,33.0,...,5.4688,32.0,5.82,100.0,91.0,50.0,35.0,2021-04-06 14:10:45,24.909687,67.079369
574276,574277,113.0,8.0,240.514435,-22.4856,0.250,61.1765,1345.0,69.4118,33.0,...,5.4688,32.0,5.82,100.0,91.0,50.0,35.0,2021-04-06 14:10:45,24.909687,67.079369
586235,586236,113.0,9.0,240.514435,-22.4856,0.855,40.3922,1170.0,54.1176,23.0,...,3.9063,32.0,3.65,100.0,91.0,50.0,35.0,2021-04-06 14:10:47,24.909687,67.079369
574278,574279,113.0,9.0,240.514435,-22.4856,0.855,40.3922,1170.0,54.1176,23.0,...,3.9063,32.0,3.65,100.0,91.0,50.0,35.0,2021-04-06 14:10:47,24.909687,67.079369


Removing duplicated entries

In [14]:
df = df.drop_duplicates(subset=['time_stamp'])
print('No. of rows after deleting duplicate entries:',len(df))

No. of rows after deleting duplicate entries: 630929


Exporting the cleaned data frame

In [77]:
df.to_csv('cleaned_data_2021.csv')